<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/llm/portkey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在Colab中打开"/></a>


# 传送门

**传送门**是一个全栈LLMOps平台，可可靠且安全地将您的Gen AI应用投入生产。


#### Portkey与Llamaindex集成的关键特性：

<img src="https://3798672042-files.gitbook.io/~/files/v0/b/gitbook-x-prod.appspot.com/o/spaces%2FeWEp2XRBGxs7C1jgAdk7%2Fuploads%2FjDGBQvw5aFOCqctr0xwp%2FColab%20Version%202.png?alt=media&token=16057c99-b86c-416c-932e-c2b71549c506" alt="header" width=600 />


1. **[🚪 AI 网关](#🔁-implementing-fallbacks-and-retries-with-portkey)**:
    - **[自动回退和重试](#🔁-implementing-fallbacks-and-retries-with-portkey)**: 确保您的应用在主要服务失败时仍保持功能性。
    - **[负载均衡](#⚖️-implementing-load-balancing-with-portkey)**: 在多个模型之间高效分发传入请求。
    - **[语义缓存](#🧠-implementing-semantic-caching-with-portkey)**: 通过智能缓存结果来降低成本和延迟。
2. **[🔬 可观测性](#🔬-observability-with-portkey)**:
    - **日志记录**: 跟踪所有请求以进行监控和调试。
    - **请求追踪**: 了解每个请求的过程以进行优化。
    - **自定义标签**: 将请求分段和分类以获得更好的洞察。
3. **[📝 用户反馈持续改进](#📝-feedback-with-portkey)**:
    - **反馈收集**: 无缝地收集任何已服务请求的反馈，无论是在生成还是对话级别。
    - **加权反馈**: 通过给用户反馈值附加权重来获得细致的信息。
    - **反馈元数据**: 将自定义元数据与反馈结合，提供上下文，从而获得更丰富的洞察和分析。
4. **[🔑 安全密钥管理](#feedback-with-portkey)**:
    - **虚拟密钥**: Portkey将原始提供者密钥转换为虚拟密钥，确保您的主要凭据保持不变。
    - **多个标识符**: 能够为同一提供者添加多个密钥，或者为同一密钥添加不同名称，以便易于识别而不影响安全性。

要利用这些功能，让我们从设置开始：

<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/docs/examples/llm/portkey.ipynb" target="_blank">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt=\"在Colab中打开\" width=150 />
</a>


如果您在colab上打开这个笔记本，您可能需要安装LlamaIndex 🦙。


In [ ]:
%pip install llama-index-llms-portkey

In [ ]:
!pip install llama-index

In [ ]:
# 安装Llamaindex & Portkey SDK!pip install -U llama_index!pip install -U portkey-ai# 导入必要的库和模块from llama_index.llms.portkey import Portkeyfrom llama_index.core.llms import ChatMessageimport portkey as pk

您不需要安装**任何**其他SDK或在您的Llamaindex应用程序中导入它们。


#### **步骤 1️⃣：获取您的Portkey API密钥和OpenAI、Anthropic等的虚拟密钥**

**[Portkey API密钥](https://app.portkey.ai/)**：在[Portkey这里](https://app.portkey.ai/)登录，然后点击左上角的个人资料图标，选择“复制API密钥”。


In [ ]:
import os

os.environ["PORTKEY_API_KEY"] = "PORTKEY_API_KEY"

**[虚拟密钥](https://docs.portkey.ai/key-features/ai-provider-keys)**
1. 转到[Portkey仪表板](https://app.portkey.ai/)上的“虚拟密钥”页面，点击位于右上角的“添加密钥”按钮。
2. 选择您的AI提供商（OpenAI、Anthropic、Cohere、HuggingFace等），为您的密钥分配一个唯一的名称，并在需要时记录任何相关的使用说明。您的虚拟密钥已准备就绪！

<img src="https://3798672042-files.gitbook.io/~/files/v0/b/gitbook-x-prod.appspot.com/o/spaces%2FeWEp2XRBGxs7C1jgAdk7%2Fuploads%2F66S1ik16Gle8jS1u6smr%2Fvirtual_keys.png?alt=media&token=2fec1c39-df4e-4c93-9549-7445a833321c" alt="header" width=600 />
3. 现在复制并粘贴下面的密钥 - 您可以在Portkey生态系统中的任何地方使用它们，并保持您的原始密钥安全和未更改。


In [ ]:
openai_virtual_key_a = ""
openai_virtual_key_b = ""

anthropic_virtual_key_a = ""
anthropic_virtual_key_b = ""

cohere_virtual_key_a = ""
cohere_virtual_key_b = ""

如果您不想使用Portkey的虚拟密钥，也可以直接使用您的AI提供商密钥。


In [ ]:
os.environ["OPENAI_API_KEY"] = ""
os.environ["ANTHROPIC_API_KEY"] = ""

#### **步骤 2️⃣：配置Portkey功能**

为了充分发挥Portkey与Llamaindex集成的潜力，您可以根据上面的示例配置各种功能。以下是所有Portkey功能及其期望值的指南：

| 功能                | 配置键                  | 值（类型）                                      | 是否必需    |
|---------------------|-------------------------|--------------------------------------------------|-------------|
| API密钥             | `api_key`               | `string`                                         | ✅ 必需（可以在外部设置） |
| 模式                | `mode`                  | `fallback`, `loadbalance`, `single`              | ✅ 必需 |
| 缓存类型            | `cache_status`          | `simple`, `semantic`                             | ❔ 可选 |
| 强制缓存刷新       | `cache_force_refresh`   | `True`, `False`                                  | ❔ 可选 |
| 缓存时效            | `cache_age`             | `integer`（以秒为单位）                           | ❔ 可选 |
| 跟踪ID             | `trace_id`              | `string`                                         | ❔ 可选 |
| 重试次数            | `retry`                 | `integer` [0,5]                                  | ❔ 可选 |
| 元数据              | `metadata`              | `json对象` [更多信息](https://docs.portkey.ai/key-features/custom-metadata)          | ❔ 可选 |
| 基本URL             | `base_url`              | `url`                                            | ❔ 可选 |

* `api_key` 和 `mode` 是必需值。
* 您可以使用Portkey构造函数设置Portkey API密钥，也可以将其设置为环境变量。
* 有 **3** 种模式 - 单一（Single）、回退（Fallback）、负载均衡（Loadbalance）。
  * **单一** - 这是标准模式。如果不需要回退或负载均衡功能，请使用此模式。
  * **回退** - 如果要启用回退功能，请设置此模式。[在此处查看指南](#implementing-fallbacks-and-retries-with-portkey)。
  * **负载均衡** - 如果要启用负载均衡功能，请设置此模式。[在此处查看指南](#implementing-load-balancing-with-portkey)。

以下是如何设置其中一些功能的示例：


In [ ]:
portkey_client = Portkey(    mode="single",)# 由于我们已经在os.environ中定义了Portkey API密钥，因此我们不需要在这里再次设置api_key

#### **步骤 3️⃣：构建LLM**

通过Portkey集成，构建LLM变得更加简单。对于所有提供者，可以使用`LLMOptions`函数，其中包含你在OpenAI或Anthropic构造函数中习惯使用的相同键。唯一的新键是`weight`，这对于负载平衡功能至关重要。


In [ ]:
openai_llm = pk.LLMOptions(
    provider="openai",
    model="gpt-4",
    virtual_key=openai_virtual_key_a,
)

上述代码演示了如何利用`LLMOptions`函数来设置一个与OpenAI提供者和GPT-4模型配合的LLM。这个函数也可以用于其他提供者，使得在不同提供者之间集成过程更加简化和一致。


#### **步骤 4️⃣：激活Portkey客户端**

一旦你使用`LLMOptions`函数构建了LLM，下一步就是使用Portkey激活它。这一步是必不可少的，以确保所有Portkey功能对你的LLM都是可用的。


In [ ]:
portkey_client.add_llms(openai_llm)

就是这样！只需4个步骤，您就可以为您的Llamaindex应用程序注入先进的生产能力。


#### **🔧 测试集成**

让我们确保一切都设置正确。下面，我们创建一个简单的聊天场景，并通过我们的Portkey客户端传递它，以查看响应。


In [ ]:
messages = [
    ChatMessage(role="system", content="You are a helpful assistant"),
    ChatMessage(role="user", content="What can you do?"),
]
print("Testing Portkey Llamaindex integration:")
response = portkey_client.chat(messages)
print(response)

以下是您的日志将在[Portkey仪表板](https://app.portkey.ai/)上显示的方式：

<img src="https://portkey.ai/blog/content/images/2023/09/Log-1.png" alt="日志" width=600 />


#### **⏩ 流式响应**

使用Portkey，流式响应变得更加简单。Portkey有4个响应函数：

1. `.complete(prompt)`
2. `.stream_complete(prompt)`
3. `.chat(messages)`
4. `.stream_chat(messages)`

`complete`函数期望一个字符串输入（`str`），而`chat`函数使用一个`ChatMessage`对象的数组。

**示例用法：**


In [ ]:
# 让我们设置一个提示，然后使用stream_complete函数来获取一个流式响应。提示 = "为什么天空是蓝色的？"print("\n测试Stream Complete：\n")响应 = portkey_client.stream_complete(提示)for i in 响应:    print(i.delta, end="", flush=True)# 让我们准备一组聊天消息，然后利用stream_chat函数来实现流式聊天响应。消息 = [    ChatMessage(role="system", content="你是一个有帮助的助手"),    ChatMessage(role="user", content="你能做什么？"),]print("\n测试Stream Chat：\n")响应 = portkey_client.stream_chat(消息)for i in 响应:    print(i.delta, end="", flush=True)

#### **🔍 总结和参考资料**

恭喜！🎉 您已成功设置并测试了Portkey与Llamaindex的集成。总结一下步骤：

1. pip install portkey-ai
2. from llama_index.llms import Portkey
3. 获取您的Portkey API密钥，并从[这里](https://app.portkey.ai/)创建您的虚拟提供者密钥。
4. 构建您的Portkey客户端并设置模式：`portkey_client=Portkey(mode="fallback")`
5. 使用LLMOptions构建您的提供者LLM：`openai_llm = pk.LLMOptions(provider="openai", model="gpt-4", virtual_key=openai_key_a)`
6. 使用`portkey_client.add_llms(openai_llm)`将LLM添加到Portkey中
7. 像使用任何其他LLM一样定期调用Portkey方法，使用`portkey_client.chat(messages)`

以下是所有函数及其参数的指南：
- [Portkey LLM 构造函数](#step-2-add-all-the-portkey-features-you-want-as-illustrated-below-by-calling-the-portkey-class)
- [LLMOptions 构造函数](https://github.com/Portkey-AI/rubeus-python-sdk/blob/4cf3e17b847225123e92f8e8467b41d082186d60/rubeus/api_resources/utils.py#L179)
- [Portkey + Llamaindex 功能列表](#portkeys-integration-with-llamaindex-adds-the-following-production-capabilities-to-your-apps-out-of-the-box)

<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/docs/examples/llm/portkey.ipynb" target="_blank">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt=\"在Colab中打开\" width=150 />
</a>


#### **🔁 使用Portkey实现回退和重试**

对于构建具有弹性的AI应用程序来说，回退和重试是至关重要的。使用Portkey，实现这些功能非常简单：

- **回退**: 如果主要服务或模型失败，Portkey将自动切换到备用模型。
- **重试**: 如果请求失败，可以配置Portkey多次重试该请求。

下面，我们将演示如何使用Portkey设置回退和重试：


In [ ]:
portkey_client = Portkey(mode="fallback")
messages = [
    ChatMessage(role="system", content="You are a helpful assistant"),
    ChatMessage(role="user", content="What can you do?"),
]

llm1 = pk.LLMOptions(
    provider="openai",
    model="gpt-4",
    retry_settings={"on_status_codes": [429, 500], "attempts": 2},
    virtual_key=openai_virtual_key_a,
)

llm2 = pk.LLMOptions(
    provider="openai",
    model="gpt-3.5-turbo",
    virtual_key=openai_virtual_key_b,
)

portkey_client.add_llms(llm_params=[llm1, llm2])

print("Testing Fallback & Retry functionality:")
response = portkey_client.chat(messages)
print(response)

#### **⚖️ 使用Portkey实现负载均衡**

负载均衡确保传入的请求在多个模型之间得到有效分发。这不仅增强了性能，还在某个模型失败时提供了冗余。

使用Portkey，实现负载均衡很简单。您需要：

- 为每个LLM定义`weight`参数。这个权重决定了请求在LLMs之间的分配情况。
- 确保所有LLMs的权重之和等于1。

以下是使用Portkey设置负载均衡的示例：


In [ ]:
portkey_client = Portkey(mode="ab_test")

messages = [
    ChatMessage(role="system", content="You are a helpful assistant"),
    ChatMessage(role="user", content="What can you do?"),
]

llm1 = pk.LLMOptions(
    provider="openai",
    model="gpt-4",
    virtual_key=openai_virtual_key_a,
    weight=0.2,
)

llm2 = pk.LLMOptions(
    provider="openai",
    model="gpt-3.5-turbo",
    virtual_key=openai_virtual_key_a,
    weight=0.8,
)

portkey_client.add_llms(llm_params=[llm1, llm2])

print("Testing Loadbalance functionality:")
response = portkey_client.chat(messages)
print(response)

#### **🧠 使用Portkey实现语义缓存**

语义缓存是一种智能缓存机制，能够理解请求的上下文。与仅基于精确输入匹配进行缓存不同，语义缓存识别相似的请求并提供缓存结果，减少冗余请求，提高响应时间，节省成本。

让我们看看如何使用Portkey实现语义缓存：


In [ ]:
import time

portkey_client = Portkey(mode="single")

openai_llm = pk.LLMOptions(
    provider="openai",
    model="gpt-3.5-turbo",
    virtual_key=openai_virtual_key_a,
    cache_status="semantic",
)

portkey_client.add_llms(openai_llm)

current_messages = [
    ChatMessage(role="system", content="You are a helpful assistant"),
    ChatMessage(role="user", content="What are the ingredients of a pizza?"),
]

print("Testing Portkey Semantic Cache:")

start = time.time()
response = portkey_client.chat(current_messages)
end = time.time() - start

print(response)
print(f"{'-'*50}\nServed in {end} seconds.\n{'-'*50}")

new_messages = [
    ChatMessage(role="system", content="You are a helpful assistant"),
    ChatMessage(role="user", content="Ingredients of pizza"),
]

print("Testing Portkey Semantic Cache:")

start = time.time()
response = portkey_client.chat(new_messages)
end = time.time() - start

print(response)
print(f"{'-'*50}\nServed in {end} seconds.\n{'-'*50}")

Portkey的缓存支持另外两个关键的缓存功能 - 强制刷新和缓存时效。

`cache_force_refresh`：强制发送一个请求到您的提供者，而不是从缓存中提供它。
`cache_age`：决定缓存存储此特定字符串应该自动刷新的时间间隔。缓存时效以秒为单位设置。

以下是您可以使用它的方式：


In [ ]:
# 将缓存状态设置为“semantic”，缓存年龄设置为60秒。openai_llm = pk.LLMOptions(    provider="openai",    model="gpt-3.5-turbo",    virtual_key=openai_virtual_key_a,    cache_force_refresh=True,    cache_age=60,)

#### **🔬 使用Portkey进行可观测性**

深入了解应用程序的行为非常重要。Portkey的可观测性功能可以让您轻松监视、调试和优化AI应用程序。您可以跟踪每个请求，了解其路径，并根据自定义标签对其进行分段。这种详细级别可以帮助识别瓶颈、优化成本并增强整体用户体验。

以下是如何使用Portkey设置可观测性：


In [ ]:
metadata = {
    "_environment": "production",
    "_prompt": "test",
    "_user": "user",
    "_organisation": "acme",
}

trace_id = "llamaindex_portkey"

portkey_client = Portkey(mode="single")

openai_llm = pk.LLMOptions(
    provider="openai",
    model="gpt-3.5-turbo",
    virtual_key=openai_virtual_key_a,
    metadata=metadata,
    trace_id=trace_id,
)

portkey_client.add_llms(openai_llm)

print("Testing Observability functionality:")
response = portkey_client.chat(messages)
print(response)

#### **🌉 开源AI网关**

Portkey的AI网关在内部使用[开源项目Rubeus](https://github.com/portkey-ai/rubeus)。Rubeus支持诸如LLM的互操作性、负载平衡、回退等功能，并充当中间人，确保您的请求得到最佳处理。

使用Portkey的优势之一是其灵活性。您可以轻松定制其行为，将请求重定向到不同的提供者，甚至完全绕过对Portkey的日志记录。

以下是使用Portkey自定义行为的示例：

```py
portkey_client.base_url=None
```


#### **📝 使用Portkey进行反馈**

持续改进是人工智能的基石。为了确保您的模型和应用程序不断发展并更好地为用户提供服务，反馈是至关重要的。Portkey的反馈API提供了一种简单的方式来收集用户的加权反馈，帮助您随着时间的推移进行改进和提升。

以下是如何利用Portkey的反馈API：

阅读有关[此处的反馈](https://docs.portkey.ai/key-features/feedback-api)。


In [ ]:
import requestsimport json# 终端URLurl = "https://api.portkey.ai/v1/feedback"# 头部信息headers = {    "x-portkey-api-key": os.environ.get("PORTKEY_API_KEY"),    "Content-Type": "application/json",}# 数据data = {"trace_id": "llamaindex_portkey", "value": 1}# 发送请求response = requests.post(url, headers=headers, data=json.dumps(data))# 打印响应print(response.text)

所有带有每个追踪ID的`weight`和`value`的反馈都可以在Portkey仪表板上找到：

<img src="https://portkey.ai/blog/content/images/2023/09/feedback.png" alt="反馈" width=600 />


#### **✅ 结论**

将Portkey与Llamaindex集成，简化了构建强大和具有韧性的人工智能应用程序的过程。通过语义缓存、可观察性、负载平衡、反馈和回退等功能，您可以确保最佳性能和持续改进。

通过遵循本指南，您已经设置并测试了Portkey与Llamaindex的集成。在继续构建和部署人工智能应用程序时，请记得充分利用这种集成的潜力！

如需进一步协助或有疑问，请联系开发人员 ➡️ <br />
<a href="https://twitter.com/intent/follow?screen_name=portkeyai" target="_blank">
  <img src="https://img.shields.io/twitter/follow/portkeyai?style=social&logo=twitter" alt="Twitter">
</a>

加入我们的从业者社区，将LLMs投入生产 ➡️ <br />
<a href="https://discord.gg/sDk9JaNfK8" target="_blank">
  <img src="https://img.shields.io/discord/1143393887742861333?logo=discord" alt="Discord">
</a>
